In [3]:
import ee
import geemap as emap

In [6]:
ee.Initialize()
ee.Authenticate()

Enter verification code: 4/1AX4XfWgKQbrLk65gB9a1i87ccVhwKNJJuABtAceWyXyWgd9aJXoNGs0uxNA

Successfully saved authorization token.


In [7]:
Map=emap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

**Load the shapefile points data and Sentinel-2 Collection**

In [8]:
point=ee.FeatureCollection("users/miketu72/Test_point")

In [9]:
Map.centerObject(point,10)
Map.addLayer(point,{},"Point")
Map

Map(bottom=1851.0, center=[21.24149464251615, 105.49841825544672], controls=(WidgetControl(options=['position'…

In [10]:
sentinel2=ee.ImageCollection("COPERNICUS/S2_SR").filterDate("2020-01-01","2020-12-31").\
filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than",30).filterBounds(point).first()

**Extract raster values from a single `sentinel2` image**

In [11]:
data=sentinel2.reduceRegion(reducer=ee.Reducer.first(),geometry=point.geometry(), scale= 10)

In [12]:
data.getInfo() # It should extract values at all points. However, only one point extracted.

{'AOT': 420,
 'B1': 901,
 'B11': 1891,
 'B12': 1257,
 'B2': 1106,
 'B3': 1616,
 'B4': 1854,
 'B5': 2313,
 'B6': 2297,
 'B7': 2354,
 'B8': 2401,
 'B8A': 2291,
 'B9': 2265,
 'MSK_CLDPRB': 27,
 'MSK_SNWPRB': 0,
 'QA10': 0,
 'QA20': 0,
 'QA60': 0,
 'SCL': 5,
 'TCI_B': 114,
 'TCI_G': 165,
 'TCI_R': 189,
 'WVP': 1935}

**Better approach for this task**

In [13]:
sampleRegions=sentinel2.sampleRegions(collection=point, scale=10,geometries =True).getInfo()

In [16]:
import pandas as pd
def data(sampleRegions):
    test=sampleRegions["features"]
    mlist=[]
    coord=[]
    for item in test:
        mdict=item["properties"]
        geom=tuple(item["geometry"]["coordinates"])
        coord.append(geom)
        mlist.append(mdict)
    test=pd.DataFrame(mlist)
    test["Coordinate"]=coord
    return test          

In [17]:
df=data(sampleRegions)
df.head()

,AOT,B1,B11,B12,B2,B3,B4,B5,B6,B7,...,MSK_SNWPRB,QA10,QA20,QA60,SCL,TCI_B,TCI_G,TCI_R,WVP,Coordinate
0,420,852,195,145,856,1210,1003,1481,1014,1017,...,0,0,0,0,6,88,122,100,1586,"(105.4664572274428, 21.257836779293218)"
1,420,1024,3339,2343,1080,1484,1990,2284,2530,2609,...,0,0,0,0,5,109,150,202,1564,"(105.48221735810253, 21.273061800349055)"
2,420,827,1276,818,921,1164,1074,1391,1634,1669,...,0,0,0,0,7,98,118,112,1586,"(105.5048508995842, 21.295763913397618)"
3,420,882,605,417,983,1394,1108,1688,1685,1699,...,0,0,0,0,6,101,143,112,1586,"(105.47630965597945, 21.29440147616462)"
4,420,710,212,144,743,912,781,1032,704,673,...,0,0,0,0,6,75,93,79,1586,"(105.46062957558976, 21.274477476785552)"


### Select images from specified date ranges

In [18]:
myBoarder=ee.FeatureCollection("users/miketu72/Test_Area")

In [19]:
def test(date):
    image = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(myBoarder).filterDate(date[0],date[-1]).filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 0.5)\
    .mean().clip(myBoarder)
    return image
dates = [('2018-06-01', '2018-12-31'),('2020-06-01', '2020-12-31'),('2019-06-01', '2019-12-31')]

In [20]:
mlist=[]

for i in dates:
    ketqua=test(i)
    mlist.append(ketqua)
col=ee.ImageCollection.fromImages(mlist) 

**Covered all in a function**

In [21]:
def test(date):
    mlist=[]
    for x in date:
            image = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(myBoarder).\
            filterDate(x[0],x[-1]).filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 0.5)\
            .mean().clip(myBoarder)
            mlist.append(image)
    return mlist
ketqua=ee.ImageCollection(test(dates))

In [22]:
# Hourly global temperature data
tem=ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate("2017-01-01","2017-01-03")
tem.size().getInfo()

48